<h1> Introduction </h1>

This notebook demonstrates how to scale an implemenation of Full Waveform Inversion using multiple FPGAs with bitstreams containing one or multiple copied compute units.

<h3> Define experiment parameters: </h3>
    
XCLBIN_PATH_DEFAULT => Default path for the .xclbin file containing 1 compute unit  
XCLBIN_PATH_MULTCU => Default path for .xclbin file containing 2 copied compute units  
XRT_ENV_PATH => Path to the Xilinx Runtime setup script.  
DEVICE_NAME_DEFAULT => Default name for the FPGA device  
DIR_PATH => Path to the directory containing the FWI input files.  

In [5]:
XCLBIN_PATH_DEFAULT = "/bitstreams/u280_xclbin/500_500_HBM/FullW.xclbin"
XCLBIN_PATH_MULTCU = "/bitstreams/u280_xclbin/500_250_HBM/FullW.xclbin"
XRT_ENV_PATH = "/opt/xilinx/xrt/setup.sh"
DEVICE_NAME_DEFAULT="xilinx_u280_xdma_201920_3"

DIR_PATH = "default/"

<h3> Define the worker method </h3> 

Here, we define the Python method which will be executed on each of the Dask workers. This function calls the driver using the data partition it receives, and returns the output data (along with some performance statistics) to the caller (the Dask client). 

We present two methods, the first can be used to execute single compute unit bitstreams, the second demonstrates how we can use multiple dask workers with the same bitstream.

In [8]:
def execute_function(queue, grid_data,kernel,path,index,config):
    import numpy as np
    import time

    start_time = time.time()

    from pynq import Overlay, allocate, Device, lib
    from FWIDriver import FWI

    resolution = config["Freq"]["nTotal"] * config["nSources"] * config["nReceivers"]
    gridsize = len(grid_data)   
    config["tolerance"] = 9.99*10**-7
    config["max"] = 1000

    # Load the overlay
    devices = Device.devices
    ol = Overlay(path, download=True, device=devices[0])

    # Allocate the buffers
    A = allocate(shape=(resolution,gridsize), dtype=np.complex64, target=getattr(ol,kernel[0]["dotprod_"+index][0]))
    B = allocate(shape=(gridsize,), dtype=np.float32, target=getattr(ol,kernel[0]["dotprod_"+index][1]))
    C = allocate(shape=(resolution,), dtype=np.complex64, target=getattr(ol,kernel[0]["dotprod_"+index][2]))

    D = allocate(shape=(resolution,gridsize), dtype=np.complex64,  target=getattr(ol,kernel[1]["update_"+index][0]))
    E = allocate(shape=(resolution,),dtype=np.complex64,  target=getattr(ol,kernel[1]["update_"+index][1]))
    F = allocate(shape=(gridsize), dtype=np.complex64, target=getattr(ol,kernel[1]["update_"+index][2]))

    # set up the kernel IP's
    dotprod = getattr(ol,"dotprod_"+index)
    update = getattr(ol,"update_"+index)


    fwi = FWI(A,B,C,D,E,F,dotprod,update,config,resolution,gridsize,True)

    # pre process the grid data
    fwi.pre_process(grid_data)

    # reconstruct the grid by performing Full Wavefrom Inversion
    chi = fwi.reconstruct()

    total_time = time.time() - start_time

    # free all the buffers
    A.freebuffer()
    B.freebuffer()
    C.freebuffer()
    D.freebuffer()
    E.freebuffer()
    F.freebuffer()
    ol.free()
    
    dict_t = {
    "index": index,
    "time": total_time,
    "chi":chi
    }
    return dict_t
    


Second method!

In [ ]:
def run_on_worker(grid_data,kernel,index):
    from dask import delayed, compute
    from pynq import Overlay, allocate, Device, lib
    from multiprocessing import Queue, Process

    devices = Device.devices
    ol = Overlay(kernel["path_to_kernel"], download=True, device=devices[0])
    
    def execute_function(queue, grid_data,kernel,path,index,config):
        import numpy as np
        import time
        
        start_time = time.time()


        from pynq import Overlay, allocate, Device, lib
        from FWIDriver import FWI
        
        
        resolution = config["Freq"]["nTotal"] * config["nSources"] * config["nReceivers"]
        gridsize = len(grid_data)   
        config["tolerance"] = 9.99*10**-7
        config["max"] = 1000

        # Load the overlay
        devices = Device.devices
        ol = Overlay(path, download=True, device=devices[0])
        
        # Allocate the buffers
        A = allocate(shape=(resolution,gridsize), dtype=np.complex64, target=getattr(ol,kernel[0]["dotprod_"+index][0]))
        B = allocate(shape=(gridsize,), dtype=np.float32, target=getattr(ol,kernel[0]["dotprod_"+index][1]))
        C = allocate(shape=(resolution,), dtype=np.complex64, target=getattr(ol,kernel[0]["dotprod_"+index][2]))

        D = allocate(shape=(resolution,gridsize), dtype=np.complex64,  target=getattr(ol,kernel[1]["update_"+index][0]))
        E = allocate(shape=(resolution,),dtype=np.complex64,  target=getattr(ol,kernel[1]["update_"+index][1]))
        F = allocate(shape=(gridsize), dtype=np.complex64, target=getattr(ol,kernel[1]["update_"+index][2]))

        # set up the kernel IP's
        dotprod = getattr(ol,"dotprod_"+index)
        update = getattr(ol,"update_"+index)
        
        
        fwi = FWI(A,B,C,D,E,F,dotprod,update,config,resolution,gridsize,True)
        
        # pre process the grid data
        fwi.pre_process(grid_data)
        
        # reconstruct the grid by performing Full Wavefrom Inversion
        chi = fwi.reconstruct()
        
        total_time = time.time() - start_time

        # free all the buffers
        A.freebuffer()
        B.freebuffer()
        C.freebuffer()
        D.freebuffer()
        E.freebuffer()
        F.freebuffer()
        
        dict_t = {
        "index": index,
        "time": total_time,
        "chi":chi
        }
        queue.put(dict_t)
    

    # Create a subprocess to handle multiple compute units, if we only use single compute units bitstreams we can map the execute functions directly
    # TODO: example of that?
    if kernel["no_instances"]==1:
        q = Queue()
        cu = 0
        p = Process(target=execute_function,args=(q, grid_data[cu],kernel["functions"][cu],kernel["path_to_kernel"],str(cu+1),kernel["config"]))
        p.start()
        result = q.get()
        p.join()
    else:
        p = []
        q = []
        result = []
        for cu in range(kernel["no_instances"]):
            q.append(Queue())
            p.append(Process(target=execute_function,args=(q[cu], grid_data[cu],kernel["functions"][cu],kernel["path_to_kernel"],str(cu+1),kernel["config"])))
            p[cu].start()
        for cu in range(kernel["no_instances"]):
            result.append(q[cu].get())
        for cu in range(kernel["no_instances"]):
            p[cu].join()
     
    # Free the overlay and return the results
    ol.free()
    return result
            


In [10]:
from Octoray import Octoray

# Create an octoray instance with the 
octoray = Octoray(ssh_cluster=True, scheduler="10.1.212.126",scheduler_port=8786, hosts=["10.1.212.127"], config_file="cluster_config.json")


In [11]:
# first load in the data
import json
import copy
import time


# Load in data and config settings
data = []
with open(DIR_PATH+"input/"+"10x10_100"+".txt") as f:
    for l in f:
        data.append(float(l))

config = None
with open(DIR_PATH+"input/GenericInput.json") as f:
    config = json.load(f)

#set specific configurations for different types of kernels
single_cu_config = config
double_cu_config = copy.deepcopy(config)
single_cu_config["ngrid"]["x"]=50
double_cu_config["ngrid"]["x"]=25

# Configure the kernels by specifying the path to the bitstream, number of compute units, batchsize per compute unit and the function names and variables with their respective memory banks.
single_cu = octoray.create_kernel(XCLBIN_PATH_DEFAULT,1,500,[[{"dotprod_1":["HBM0","HBM1","HBM2"]},{"update_1":["HBM3","HBM4","HBM5"]}]],single_cu_config)

double_cu = octoray.create_kernel(XCLBIN_PATH_MULTCU,2,250,[[{"dotprod_1":["HBM0","HBM1","HBM2"]},{"update_1":["HBM6","HBM7","HBM8"]}],
                                                [{"dotprod_2":["HBM3","HBM4","HBM5"]},{"update_2":["HBM9","HBM10","HBM11"]}]],double_cu_config)

# Finally, add the kernels you want to execute
data_split, kernels_split = octoray.setup_cluster(data,single_cu)

# Execute the added kernels on the workers.
results = octoray.execute_function(execute_function,data_split,kernels_split)

print(results)

Initializing OctoRay with client ip: 10.1.212.126


distributed.deploy.ssh - INFO - distributed.scheduler - INFO - Clear task state
distributed.deploy.ssh - INFO - distributed.scheduler - INFO -   Scheduler at:   tcp://10.1.212.126:8786
distributed.deploy.ssh - INFO - distributed.utils - INFO - Reload module pynqimport from .py file
distributed.deploy.ssh - INFO - distributed.preloading - INFO - Import preload module: pynqimport.py
distributed.deploy.ssh - INFO - distributed.worker - INFO -       Start worker at:   tcp://10.1.212.126:41381
Task exception was never retrieved
future: <Task finished coro=<_wrap_awaitable() done, defined at /opt/tools/external/anaconda/envs/pynq-dask/lib/python3.7/asyncio/tasks.py:623> exception=PermissionDenied('Permission denied')>
Traceback (most recent call last):
  File "/opt/tools/external/anaconda/envs/pynq-dask/lib/python3.7/asyncio/tasks.py", line 630, in _wrap_awaitable
    return (yield from awaitable.__await__())
  File "/home/ldierick/.local/lib/python3.7/site-packages/distributed/deploy/spec.p

PermissionDenied: Permission denied

<h3> Graceful shutdown for OpenSSH version >= 7.9 </h3>

In [ ]:
octoray.shutdown()

<h3> Ungraceful shutdown for OpenSSH version < 7.9 </h3>

In [ ]:
await octoray.fshutdown()